
<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3: Linear Regression and Random Forests - Train/Test Split

---

# Introduction

We've discussed overfitting in the context of bias and variance, and we've touched on some techniques, such as regularization, that are used to avoid overfitting (but haven't practiced them yet). In this lesson we'll discuss a fundamental method for avoiding overfitting that is commonly referred to as _train/test split_ validation. 

The idea is similar to something called "cross-validation" — in fact, it is a type of cross-validation — in that we split the data set into two subsets:
* A subset on which to train our model.
* A subset on which to test our model's predictions.

This serves two useful purposes:
* We prevent overfitting by not using all of the data.
* We have some remaining data we can use to evaluate our model.

While this may seem like a relatively simple idea, **there are some caveats** to putting it into practice. For example, if you are not careful, it is easy to take a non-random split. Suppose we have salary data on technical professionals that is composed of 80 percent data from California and 20 percent data from elsewhere and is sorted by state. If we split our data into 80 percent training data and 20 percent testing data, we might inadvertantly select all the California data to train and all the non-California data to test. In this case we've still overfit on our data set because we did not sufficiently randomize the data.

In a situation like this we can use _k-fold cross-validation_, which is the same idea applied to more than two subsets. In particular, we partition our data into $k$ subsets and train on $k-1$ one of them, holding the last slice for testing. We can do this for each of the possible $k-1$ subsets.

# Independent Practice

Ultimately we use a test-training split to compare multiple models on the same data set. This could be comparisons of two linear models or of completely different models on the same data.

For your independent practice, fit three different models on the Boston housing data. For example, you could pick three different subsets of variables, one or more polynomial models, or any other model you'd like. 

### Here's What We Will Be Doing:

* Working with Boston housing data to predict the value of a home
* Create a test-train split of the data.
* Train each of your models on the training data.
* Evaluate each of the models on the test data.
* Rank the models by how well they score on the testing data set.

**Then, try k-folds.**

* Try a few different splits of data for the same models.
* Perform a k-fold cross-validation and use the cross-validation scores to compare your models. Did this change your rankings?

**Be sure to provide interpretation for your results.**

Recall that k-fold cross-validation creates a hold portion of your data set for each iteration of training and validating:

![](http://i.imgur.com/0PFrPXJ.png)

## Linear Regression Use Case

In this given task, you will be asked to model the median home price of various houses across U.S. Census tracts in the city of Boston. This is a probable use case: We are predicting a continuous, numeric output (price) based on a combination of discrete features.

In [1]:
import matplotlib.pyplot as plt

% matplotlib inline

UsageError: Line magic function `%` not found.


In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

boston = load_boston()

X = pd.DataFrame(boston.data,
                 columns=boston.feature_names)
y = pd.DataFrame(boston.target,
                 columns=['MEDV'])

print(boston['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

### 1. Clean Up Data and Perform Exporatory Data Analysis

Boston data is from scikit-learn, so it ought to be pretty clean, but we should always perform exploratory data analysis.

In [3]:
# Exploratory data analysis.

# Include: total nulls, index, data types, shape, summary statistics, and the number of unique values for each column


In [4]:
# Understand number of rows / columns in datasets

print("X = ", X.shape)
print("y = ", y.shape)

X =  (506, 13)
y =  (506, 1)


In [5]:
# Test whether there are nulls in dataset.

np.sum(X.isnull())

# Result there are no nulls in dataset

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

In [6]:
# Checking the data types of each column in dataset

X.dtypes

# Result: All columns are numeric

CRIM       float64
ZN         float64
INDUS      float64
CHAS       float64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD        float64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
dtype: object

In [7]:
#  Checking counts of unqiue values in each column. Goal to identify non-continuous variables

X.nunique()

# CHAS and RAD are not contiuous cariables. CHAS is a binary vairable and RAD is ordinal

CRIM       504
ZN          26
INDUS       76
CHAS         2
NOX         81
RM         446
AGE        356
DIS        412
RAD          9
TAX         66
PTRATIO     46
B          357
LSTAT      455
dtype: int64

## Using `scikit-learn` Linear Regression

### 2. Pick 3-4 predictors (i.e. CRIM, ZN, etc...) that you will use to predict our target variable, MEDV.
Score and plot your predictions. What do these results tell us?

In [8]:
# Testing correlation of X variables with MEDV to select 3-4 predictors

boston = pd.concat([X,y],axis=1)
boston.corr()['MEDV'].sort_values()

# Result: Variables with highest correlation are LSTAT, RM, PTRATIO, INDUS

LSTAT     -0.737663
PTRATIO   -0.507787
INDUS     -0.483725
TAX       -0.468536
NOX       -0.427321
CRIM      -0.388305
RAD       -0.381626
AGE       -0.376955
CHAS       0.175260
DIS        0.249929
B          0.333461
ZN         0.360445
RM         0.695360
MEDV       1.000000
Name: MEDV, dtype: float64

In [9]:
# Testing the correlation 

X[['LSTAT', 'RM', 'PTRATIO', 'INDUS']].corr()

# LSTAT is fairly correlated with RM and INDUS. Overall X variables are not extremely correlated and will be tested as predictors.

,LSTAT,RM,PTRATIO,INDUS
LSTAT,1.000000,-0.613808,0.374044,0.603800
RM,-0.613808,1.000000,-0.355501,-0.391676
PTRATIO,0.374044,-0.355501,1.000000,0.383248
INDUS,0.603800,-0.391676,0.383248,1.000000


In [10]:
# Create Train and Test sets

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X[['LSTAT', 'RM', 'PTRATIO', 'INDUS']], y, random_state=2020)

print("X_Train= ",X_train.shape)
print("y_Train= ",y_train.shape)
print("X_Test= ",X_test.shape)
print("y_Test= ",y_test.shape)

X_Train=  (379, 4)
y_Train=  (379, 1)
X_Test=  (127, 4)
y_Test=  (127, 1)


In [11]:
# Standardize X Training and Test

# Define mean and standard deviation to use on training and test set
train_means = X_train.mean()
train_stds  = X_train.std()

# standardize the training set
X_train_std = X_train - train_means
X_train_std /= train_stds

# and do the same for the test set
X_test -= train_means
X_test /= train_stds

# Score and plot 

lreg = LinearRegression()
lreg.fit(X_train_std, y_train)
lreg.score(X_test, y_test)

# Score is .6597

0.6597915804955761

### 3. Try 70/30 and 90/10 train/test splits (70% of the data for training - 30% for testing, then 90% for training - 10% for testing)
Score and plot. How do your metrics change? What does this tell us about the size of training/testing splits?

In [12]:
# Testing with 70% train and 30% test.

X_train, X_test, y_train, y_test = train_test_split(X[['LSTAT', 'RM', 'PTRATIO', 'INDUS']], y,test_size=.3 ,random_state=2020)

# Standardize X Training and Test

# Define mean and standard deviation to use on training and test set
train_means = X_train.mean()
train_stds  = X_train.std()

# standardize the training set
X_train_std = X_train - train_means
X_train_std /= train_stds

# and do the same for the test set
X_test -= train_means
X_test /= train_stds

# Score and plot 

lreg = LinearRegression()
lreg.fit(X_train_std, y_train)
lreg.score(X_test, y_test)

# Score is .6349

0.6349572592261994

In [13]:
# Testing with 90% train and 10% test.

X_train, X_test, y_train, y_test = train_test_split(X[['LSTAT', 'RM', 'PTRATIO', 'INDUS']], y,test_size=.1 ,random_state=2020)

# Standardize X Training and Test

# Define mean and standard deviation to use on training and test set
train_means = X_train.mean()
train_stds  = X_train.std()

# standardize the training set
X_train_std = X_train - train_means
X_train_std /= train_stds

# and do the same for the test set
X_test -= train_means
X_test /= train_stds

# Score and plot 

lreg = LinearRegression()
lreg.fit(X_train_std, y_train)
lreg.score(X_test, y_test)

# Score is .6603
# Thus score improved by decreasing the size of the test set.

0.6603565457791117

### 4. Use k-fold cross validation varying the number of folds from 5 to 10
What seems optimal? How do your scores change? What is the variance like? Try different folds to get a sense of how this impacts your score. What are the tradeoffs associated with choosing the number of folds?

In [14]:
# Use KFold to find your validation score

from sklearn.model_selection import cross_val_score

# we'll use a loop to go through these
cv_scores = []
num_folds = [5, 10]

for fold in num_folds:
    scores = cross_val_score(estimator=lreg, X=X_train_std, y=y_train, cv=fold)
    cv_scores.append(scores)
    
cv_scores

[array([0.66483586, 0.60359389, 0.48669311, 0.73382352, 0.79289127]),
 array([0.66134375, 0.66713359, 0.56387986, 0.61393163, 0.65464383,
        0.34985327, 0.73871658, 0.71765115, 0.82363217, 0.77947662])]

In [15]:
print("5 Folds Mean: ",np.mean(cv_scores[0]))
print("5 Folds Min: ",np.min(cv_scores[0]))
print("10 Folds Mean: ",np.mean(cv_scores[1]))
print("10 Folds Min: ",np.min(cv_scores[1]))

# While 10 fold has a slightly higher score it has a larger range, thus 5 folds seems more optimal.

5 Folds Mean:  0.6563675284480988
5 Folds Min:  0.4866931086815816
10 Folds Mean:  0.6570262460120486
10 Folds Min:  0.34985327120484233


  ##  Mike adding section to test with Ridge

In [16]:
# Rebuilding train and test set to incorporate all variables

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.1 ,random_state=2020)

# Standardize X Training and Test

# Define mean and standard deviation to use on training and test set
train_means = X_train.mean()
train_stds  = X_train.std()

# standardize the training set
X_train_std = X_train - train_means
X_train_std /= train_stds

# and do the same for the test set
X_test -= train_means
X_test /= train_stds

# Score and plot 

X_train_std.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
289,-0.435910,1.782441,-0.833462,-0.274743,-1.291008,0.393812,-1.625233,1.674581,-0.405469,-0.663828,-0.837170,0.151275,-0.437670
102,-0.411232,-0.483615,-0.363418,-0.274743,-0.293324,0.169819,0.602560,-0.514753,-0.520559,-0.128262,1.137183,-3.166141,-0.278134
488,-0.421579,-0.483615,2.419124,-0.274743,0.478797,-1.161538,0.862766,-0.939919,-0.635649,1.796246,0.769861,0.408911,0.780210
18,-0.334996,-0.483615,-0.424350,-0.274743,-0.137165,-1.158738,-1.136901,-0.000158,-0.635649,-0.581433,1.183098,-0.760761,-0.127146
151,-0.242865,-0.483615,1.235310,-0.274743,2.751781,-1.231536,1.122972,-1.048993,-0.520559,-0.016440,-1.709558,-0.180776,0.099337


In [17]:
# Run Ridge Regression

from sklearn.linear_model import Ridge, Lasso
ridge = Ridge()

# Test various Alpha levels to determine which returns optimal score
alphas = np.logspace(-3,3,7)

cv_scores=[]

for alpha in alphas:
    ridge.set_params(alpha=alpha)
    scores = cross_val_score(estimator=ridge, X=X_train_std, y=np.log(y_train), cv=10)
    cv_scores.append((np.mean(scores),alpha))
    
max(cv_scores)

# Result: 1 returns highest score (78.8%)

(0.7581165420849134, 1.0)

In [18]:
# Run Ridge with Alpha of 1.0 and review results

rreg = Ridge(alpha=1.0)
rreg.fit(X_train_std, np.log(y_train))

coefs = rreg.coef_

coefs_list = coefs[0]

col_list = X_train_std.columns.to_list()

coef_dict = {'Predictor':col_list, 'Coefficients':coefs_list }

coef_df = pd.DataFrame(coef_dict)
coef_df

# Result: This model most heavily weights LSTAT, RAD, DIS, Tax

,Predictor,Coefficients
0,CRIM,-0.084838
1,ZN,0.026631
2,INDUS,0.009928
3,CHAS,0.024862
4,NOX,-0.081605
5,RM,0.062723
6,AGE,0.002890
7,DIS,-0.101941
8,RAD,0.124369
9,TAX,-0.098922


In [22]:
rreg.score(X_test, np.log(y_test))

# This appraoach results in score of 80.88% which is higher than normal linear regression (66%)

0.8087772457560763

In [23]:
# Predict Results on test set


df_predict = pd.concat([X_test,y_test],axis=1)
df_predict['MEDV_predict'] = np.exp(rreg.predict(X_test))

df_predict


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,MEDV_predict
409,1.476182,-0.483615,1.020599,-0.274743,0.374691,0.795599,1.122972,-1.108977,1.666143,1.531405,0.815776,-1.969349,1.025210,27.5,15.620402
247,-0.415507,0.465970,-0.755121,-0.274743,-1.065444,-0.080773,0.381563,2.025778,-0.290380,-0.446070,0.310709,0.200002,-0.346507,20.5,20.404778
399,0.875570,-0.483615,1.020599,-0.274743,1.207540,-0.604356,0.331660,-1.092376,1.666143,1.531405,0.815776,-0.218699,2.476694,6.3,11.251188
300,-0.435750,2.537793,-1.280293,-0.274743,-1.334385,0.822198,-0.751938,1.917465,-0.520559,-0.281281,-1.663643,0.362279,-0.927670,24.8,29.942523
321,-0.417497,-0.483615,-0.534607,-0.274743,-0.527563,0.129220,-0.505990,0.353703,-0.520559,-0.699140,0.540285,0.428865,-0.813717,23.1,24.774775
31,-0.261674,-0.483615,-0.424350,-0.274743,-0.137165,-0.296366,1.122972,0.179888,-0.635649,-0.581433,1.183098,0.206506,0.065151,14.5,18.102665
342,-0.438299,-0.483615,-1.331070,-0.274743,-0.310675,0.358813,-0.313509,1.174970,-0.980917,0.095381,-1.158576,0.352357,-0.560170,16.5,21.575208
375,2.163005,-0.483615,1.020599,-0.274743,1.016678,1.440978,1.048118,-1.179949,1.666143,1.531405,0.815776,0.428865,0.122128,15.0,19.375041
379,1.931567,-0.483615,1.020599,-0.274743,1.016678,-0.084973,1.122972,-1.146746,1.666143,1.531405,0.815776,0.394029,1.310094,10.2,14.089632
288,-0.435521,1.782441,-0.833462,-0.274743,-1.291008,0.043823,-0.816099,1.674581,-0.405469,-0.663828,-0.837170,0.428865,-0.709734,22.3,27.029747


## Using Random Forests With the Boston Dataset

#### Create X and y variables for Your Data

#### Divide it into a training and test set

#### Fit a Random Forest on the data

#### What are its most important features?

#### How well does your model perform on your test set?

#### Challenge:  Try and find at least two improvements to your model to improve test scores.

You can try the following:
 - increasing the number of trees
 - using a different number of maximum features to sample
 - using a different number of minimum samples per leaf

### Example: Using the Statsmodels Formula

Adapt the formula example using your metrics. We will review this implementation in class. Here is a reference to consider. The workflow is the same, but the syntax is a little different. We want to get accustomed to the formula syntax because we will be using them a lot more with regressions. The results should be comparable to scikit-learn's regression models.

In [ ]:
# First, format our data in a DataFrame

df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['MEDV'] = boston.target
df.head()

In [ ]:
# Set up our new statsmodel.formula handling model
import statsmodels.formula.api as smf

# You can easily swap these out to test multiple versions/different formulas
formulas = {
    "case1": "MEDV ~ RM + LSTAT + RAD + TAX + NOX + INDUS + CRIM + ZN - 1", # - 1 = remove intercept
    "case2": "MEDV ~ NOX + RM",
    "case3": "MEDV ~ RAD + TAX"
}

model = smf.ols(formula=formulas['case1'], data=df)
result = model.fit()

result.summary()

### Bonus Challenge #1:

Can you optimize your R2, selecting the best features and using either test-train split or k-folds?

### Bonus Challenge #2:

Given a combination of predictors, can you find another response variable that can be accurately predicted through the exploration of different predictors in this data set?

_Tip: Check out pairplots, coefficients, and Pearson scores._

In [ ]:
# Check out variable relations
import seaborn as sns

In [ ]:
# Check out Pearson scores
